由[https://github.com/s-gupta/v-coco](https://github.com/s-gupta/v-coco)提供的标注格式，转换为COCO标注格式

In [1]:
from pprint import pprint
import os
import shutil
import numpy as np
from pycocotools.coco import COCO
from mmengine import load, dump
from utils import gen_new_coco

In [2]:
raw_rel_train = load('../../annotations/raw/vcoco_trainval.json')
raw_rel_test  = load('../../annotations/raw/vcoco_test.json')

In [3]:
raw_rel_test[0].keys()
# 所有raw_ann的image_id&ann_id都是一样的
# image_id       - Nx1, 所有有verb的img_ids
# ann_id         - Nx1, 某个潜在verb实例涉及的sub_ins_id
# label          - Nx1, 某个潜在verb实例是否属于当前action类
# action_name    - string
# role_name      - ['agent', 'obj', 'instr']
# role_object_id - N x K matrix, obviously [:,0] is same as ann_id

dict_keys(['image_id', 'ann_id', 'role_object_id', 'label', 'action_name', 'role_name', 'include'])

## image

In [3]:
def collect_images(file_names: list, src_root: str, dst_root: str):
    if not os.path.exists(dst_root):
        os.makedirs(dst_root)
    for filename in file_names:
        src_path = os.path.join(src_root, filename)
        dst_path = os.path.join(dst_root, filename)
        if not os.path.isfile(dst_path):
            shutil.copy(src_path, dst_root)

In [4]:
train_img_ids = sorted(raw_rel_train[0]['image_id'])
test_img_ids  = sorted( raw_rel_test[0]['image_id'])
train_img_filenames = [f'COCO_train2014_{id:0>12}.jpg' for id in train_img_ids]
test_img_filenames  = [f'COCO_val2014_{  id:0>12}.jpg' for id in  test_img_ids]

In [5]:
collect_images(train_img_filenames, '/data0/DataSet/COCO2014/train2014/', '../../images/train/')
collect_images( test_img_filenames, '/data0/DataSet/COCO2014/val2014/'  , '../../images/test/' )

## annotation

### instance

In [7]:
coco_ins_train = COCO('/data0/DataSet/COCO2014/annotations/instances_train2014.json')
coco_ins_test  = COCO('/data0/DataSet/COCO2014/annotations/instances_val2014.json')
coco_ins_train = gen_new_coco(coco_ins_train, valid_img_ids=raw_rel_train[0]['image_id'])
coco_ins_test  = gen_new_coco(coco_ins_test , valid_img_ids=raw_rel_test [0]['image_id'])

loading annotations into memory...
Done (t=9.53s)
creating index...
index created!
creating index...
index created!
loading annotations into memory...
Done (t=5.39s)
creating index...
index created!
creating index...
index created!


In [8]:
len(coco_ins_train.dataset['images']), len(coco_ins_test.dataset['images'])

(5400, 4946)

In [3]:
dump(dict(info=None, licenses=[], images=coco_ins_train.dataset['images']), 'image_info_train.json')
dump(coco_ins_train.dataset, 'instances_train.json')
dump(dict(info=None, licenses=[], images=coco_ins_test.dataset['images']), 'image_info_test.json')
dump(coco_ins_test.dataset, 'instances_test.json')

### person_keypoints

In [3]:
coco_key_train = COCO('/data0/DataSet/COCO2014/annotations/person_keypoints_train2014.json')
coco_key_test  = COCO('/data0/DataSet/COCO2014/annotations/person_keypoints_val2014.json')
coco_key_train = gen_new_coco(coco_key_train, valid_img_ids=raw_rel_train[0]['image_id'])
coco_key_test  = gen_new_coco(coco_key_test , valid_img_ids=raw_rel_test [0]['image_id'])

loading annotations into memory...
Done (t=5.37s)
creating index...
index created!
loading annotations into memory...
Done (t=3.19s)
creating index...
index created!
creating index...
index created!
creating index...
index created!


In [5]:
len(coco_key_train.dataset['images']), len(coco_key_test.dataset['images'])

(5400, 4946)

In [ ]:
# coco_ins_train = COCO('../../annotations/instances_train.json')
# coco_ins_test  = COCO('../../annotations/instances_test.json')
# coco_key_train = COCO('../../annotations/person_keypoints_train.json')
# coco_key_test  = COCO('../../annotations/person_keypoints_test.json')

# for ann in coco_key_test.anns.values():
#     assert ann['bbox'] == coco_ins_test.anns[ann['id']]['bbox']
# for ann in coco_key_train.anns.values():
#     assert ann['bbox'] == coco_ins_train.anns[ann['id']]['bbox']

In [6]:
dump(coco_key_train.dataset, 'person_keypoints_train.json')
dump(coco_key_test.dataset , 'person_keypoints_test.json')

### img_captions

In [7]:
coco_cap_train = COCO('/data0/DataSet/COCO2014/annotations/captions_train2014.json')
coco_cap_test  = COCO('/data0/DataSet/COCO2014/annotations/captions_val2014.json')
coco_cap_train = gen_new_coco(coco_cap_train, valid_img_ids=raw_rel_train[0]['image_id'])
coco_cap_test  = gen_new_coco(coco_cap_test , valid_img_ids=raw_rel_test [0]['image_id'])

loading annotations into memory...
Done (t=0.57s)
creating index...
index created!
loading annotations into memory...
Done (t=0.42s)
creating index...
index created!
creating index...
index created!
creating index...
index created!


In [8]:
len(coco_cap_train.dataset['images']), len(coco_cap_test.dataset['images'])

(5400, 4946)

In [9]:
dump(coco_cap_train.dataset, 'captions_train.json')
dump(coco_cap_test.dataset , 'captions_test.json')

### relation

In [3]:
coco_ins_train = COCO('../../annotations/instances_train.json')
coco_ins_test  = COCO('../../annotations/instances_test.json')

loading annotations into memory...
Done (t=1.03s)
creating index...
index created!
loading annotations into memory...
Done (t=0.75s)
creating index...
index created!


In [4]:
# rel_categories = load('rel_categories.json')
rel_categories = load('../../configs/rel_categories.json')

In [5]:
# label_info = load('label_info.json')
label_info = load('../../configs/label_info.json')
rel_info = label_info['rel_info']

In [7]:
def collect_coco_format(raw_anns: list, coco_ins: COCO,
                        keep_no_obj: bool = True) -> dict:
    rel_annotations = []
    action_names = []
    valid_img_ids = set()

    for action in raw_anns:
        roles = action['role_name']
        if len(roles) == 1:
            action_names.append(action['action_name'])
            continue  # 没有交互对象，不是交互类型的verb

        this_rels = []
        for role in roles[1:]:
            this_rels.append(len(action_names)+1)  # 1-base
            action_names.append(action['action_name']+'-'+role)
        role_ins_ids = np.array(action['role_object_id']).reshape(len(roles), -1).T.tolist()

        for img_id, isvalid, ins_ids in zip(action['image_id'], action['label'], role_ins_ids):
            if not isvalid:
                continue

            sub_id = ins_ids[0]
            assert sub_id in coco_ins.anns

            for obj_id, rel in zip(ins_ids[1:], this_rels):
                if (str(rel) not in rel_info) and (rel not in rel_info):
                    continue
                if obj_id == 0:
                    if keep_no_obj:
                        obj_id = -1
                    else:
                        continue
                else:
                    assert obj_id in coco_ins.anns
                rel_annotations.append({
                    'id'         : len(rel_annotations) + 1,  # 1-base
                    'subject_id' : sub_id,
                    'object_id'  : obj_id,
                    'category_id': rel,  # 1-base
                    'image_id'   : img_id
                })
                valid_img_ids.add(img_id)

    valid_imgs = []
    for image in coco_ins.dataset['images']:
        if image['id'] in valid_img_ids:
            valid_imgs.append(image)
    print('num total images: ', len(coco_ins.dataset['images']))
    print('num valid images: ', len(valid_imgs))

    new_rel_anns = {
        'info'       : coco_ins.dataset['info'    ],
        'licenses'   : coco_ins.dataset['licenses'],
        'images'     : valid_imgs,
        'annotations': rel_annotations,
        'categories' : rel_categories
    }
    return new_rel_anns

In [8]:
coco_rel_train = collect_coco_format(
    raw_rel_train, coco_ins_train, keep_no_obj=True)
coco_rel_test = collect_coco_format(
    raw_rel_test, coco_ins_test, keep_no_obj=True)

num total images:  5400
num valid images:  5267
num total images:  4946
num valid images:  4801


In [15]:
dump(coco_rel_train, 'relations_train.json')
dump(coco_rel_test, 'relations_test.json')